### 평범한 연습장입니다.

In [70]:
from transformers import AutoProcessor, AutoModel
import requests
import torch
from PIL import Image
processor = AutoProcessor.from_pretrained("koclip/koclip-base-pt")
model = AutoModel.from_pretrained("koclip/koclip-base-pt")
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = ["소파 와 고양이", "강아지와 강아지 주인", "쳇바퀴를 달리는 햄스터", "자동차", "뭘봐"]
image
#Load CLIP model
# model = SentenceTransformer('clip-ViT-B-32')
softmax = torch.nn.Softmax(dim = 1)
inputs = processor(
    text=text,
    images=image, 
    return_tensors="pt", # could also be "pt" 
    padding=True
)
outputs = model(**inputs)

probs = softmax(outputs.logits_per_image)

for idx, prob in sorted(enumerate(*probs), key=lambda x: x[1], reverse=True):
    print(text[idx], prob)
    
te = outputs.text_embeds
ie = outputs.image_embeds

#Encode an image:
# img_emb = model.encode(Image.open('two_dogs_in_snow.png')) # or jpg

#Encode text descriptions
# text_emb = model.encode(['두 개가 눈 위에 있다.', '고양이가 책상에 앉아있다.', '저녁 런던의 사진'])

#Compute cosine similarities 
# cos_scores = util.cos_sim(img_emb, text_emb)
# print(cos_scores)
print(te)

소파 와 고양이 tensor(0.7725, grad_fn=<UnbindBackward0>)
자동차 tensor(0.0967, grad_fn=<UnbindBackward0>)
뭘봐 tensor(0.0662, grad_fn=<UnbindBackward0>)
쳇바퀴를 달리는 햄스터 tensor(0.0609, grad_fn=<UnbindBackward0>)
강아지와 강아지 주인 tensor(0.0037, grad_fn=<UnbindBackward0>)
tensor([[ 0.0799,  0.0500, -0.0342,  ...,  0.0703,  0.0370,  0.0331],
        [-0.0421, -0.0086, -0.0381,  ..., -0.0666, -0.0429, -0.0286],
        [ 0.0690,  0.0496, -0.0178,  ..., -0.0152, -0.0364,  0.0090],
        [ 0.0620,  0.0566, -0.0062,  ..., -0.0022, -0.0360,  0.0009],
        [-0.0601,  0.0624, -0.0363,  ...,  0.0032, -0.0540, -0.0414]],
       grad_fn=<DivBackward0>)


In [74]:
from transformers import AutoProcessor, AutoModel
import requests
import torch
from PIL import Image
processor = AutoProcessor.from_pretrained("koclip/koclip-base-pt")
model = AutoModel.from_pretrained("koclip/koclip-base-pt")
url = "/Users/moohosong/Desktop/Repos/patent_search/frontend/image/2023-11-28T11_43_05.562722.jpg"
image = Image.open(url)
text = ["소파 와 고양이", "강아지와 강아지 주인", "쳇바퀴를 달리는 햄스터", "자동차", "뭘봐"]
image
#Load CLIP model
# model = SentenceTransformer('clip-ViT-B-32')
softmax = torch.nn.Softmax(dim = 1)
inputs = processor(
    text=text,
    images=image, 
    return_tensors="pt", # could also be "pt" 
    padding=True
)
outputs = model(**inputs)

probs = softmax(outputs.logits_per_image)

for idx, prob in sorted(enumerate(*probs), key=lambda x: x[1], reverse=True):
    print(text[idx], prob)
    
te = outputs.text_embeds
ie = outputs.image_embeds

#Encode an image:
# img_emb = model.encode(Image.open('two_dogs_in_snow.png')) # or jpg

#Encode text descriptions
# text_emb = model.encode(['두 개가 눈 위에 있다.', '고양이가 책상에 앉아있다.', '저녁 런던의 사진'])

#Compute cosine similarities 
# cos_scores = util.cos_sim(img_emb, text_emb)
# print(cos_scores)
print(te)

자동차 tensor(0.3735, grad_fn=<UnbindBackward0>)
강아지와 강아지 주인 tensor(0.2243, grad_fn=<UnbindBackward0>)
소파 와 고양이 tensor(0.2202, grad_fn=<UnbindBackward0>)
쳇바퀴를 달리는 햄스터 tensor(0.1324, grad_fn=<UnbindBackward0>)
뭘봐 tensor(0.0496, grad_fn=<UnbindBackward0>)
tensor([[ 0.0799,  0.0500, -0.0342,  ...,  0.0703,  0.0370,  0.0331],
        [-0.0421, -0.0086, -0.0381,  ..., -0.0666, -0.0429, -0.0286],
        [ 0.0690,  0.0496, -0.0178,  ..., -0.0152, -0.0364,  0.0090],
        [ 0.0620,  0.0566, -0.0062,  ..., -0.0022, -0.0360,  0.0009],
        [-0.0601,  0.0624, -0.0363,  ...,  0.0032, -0.0540, -0.0414]],
       grad_fn=<DivBackward0>)


In [59]:
vo = vision_output[0]
vo.shape

torch.Size([1, 50, 768])

In [60]:
to = text_output[1]
to.shape

torch.Size([5, 1024])

In [61]:
ieo = image_embeds_output
ieo.shape

torch.Size([1, 512])

In [62]:
te = outputs.text_embeds
te.shape

torch.Size([5, 512])

In [63]:
ie = outputs.image_embeds
ie.shape

torch.Size([1, 512])

In [64]:
dot = (ie * te).sum(dim = -1)
softmax0 = torch.nn.Softmax(dim = 0)
print(softmax0(dot))

tensor([0.3001, 0.1135, 0.1890, 0.2055, 0.1919], grad_fn=<SoftmaxBackward0>)


In [65]:
normed_dot = dot / (torch.norm(te, dim = -1) * torch.norm(ie, dim = -1)).reshape(-1)
print(softmax0(normed_dot))

tensor([0.3001, 0.1135, 0.1890, 0.2055, 0.1919], grad_fn=<SoftmaxBackward0>)


In [68]:
inputs = processor(
    text=text,
    return_tensors="pt", # could also be "pt" 
    padding=True
)
outputs = model(**inputs)


ValueError: You have to specify pixel_values